Vehicle Insurance Cost Estimator

1. Import Libraries

In [ ]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTImageProcessor
import torch
from torch import nn, optim
import tqdm as tqdm

2. Define Custom Dataset

In [8]:
df = pd.read_csv("data/train/train.csv")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=False)

# Save (optional)
train_df.to_csv('train_split.csv', index=False)
test_df.to_csv('test_split.csv', index=False)


3. Feature Extraction

In [9]:
feature_extractor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
])

4. Dataset Class

In [10]:
class CarDamageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.loc[idx, 'filename']
        label = int(self.df.loc[idx, 'label']) - 1  # convert to 0–5
        image_path = os.path.join(self.img_dir, img_name)
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label


    5. DataLoaders

In [11]:
train_dataset = CarDamageDataset(train_df, img_dir="data/train/images", transform=transform)
val_df = test_df.copy()
val_dataset = CarDamageDataset(val_df, img_dir="data/train/images", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class_names = ['crack', 'scratch', 'tire flat', 'dent', 'glass shatter', 'lamp broken']


6. Define the Model

In [13]:
from transformers import ViTForImageClassification

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=6 ,
    ignore_mismatched_sizes=True # since you have 6 classes
)
#optimizer, loss function
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([6]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([6, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


7. Model Training

In [22]:
from tqdm import tqdm
from transformers import get_scheduler

num_epochs = 10
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=len(train_loader) * num_epochs
) 

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Wrap train_loader with tqdm for progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False)

    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=images).logits
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        running_loss += loss.item()

        # Update tqdm postfix to show loss dynamically
        loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")



Epoch 1, Loss: 0.0003


Epoch 2, Loss: 0.0002


Epoch 3, Loss: 0.0002


Epoch 4, Loss: 0.0001


Epoch 5, Loss: 0.0001


Epoch 6, Loss: 0.0001


Epoch 7, Loss: 0.0001


Epoch 8, Loss: 0.0001


Epoch 9, Loss: 0.0001


Epoch 10, Loss: 0.0001


8. Model Evaluation and Save Predictions

In [23]:
from sklearn.metrics import classification_report
model.eval()

all_preds = []
all_labels = []
all_filenames = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=images).logits
        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Map back to original file names (order matches `test_df` due to DataLoader order)
all_filenames = test_df['filename'].values.tolist()

# Save predictions to CSV
results_df = pd.DataFrame({
    'filename': all_filenames,
    'true_label': all_labels,
    'predicted_label': all_preds
})

results_df.to_csv("val_predictions.csv", index=False)

# Calculate accuracy (optional print)
accuracy = (results_df['true_label'] == results_df['predicted_label']).mean() * 100
print(f"Validation Accuracy: {accuracy:.2f}%")
print(classification_report(all_labels, all_preds, target_names=class_names))

Validation Accuracy: 97.92%
               precision    recall  f1-score   support

        crack       0.90      1.00      0.95        35
      scratch       0.98      0.97      0.98       474
    tire flat       1.00      0.96      0.98       104
         dent       0.97      0.99      0.98       413
glass shatter       1.00      0.99      0.99       233
  lamp broken       0.98      0.98      0.98       181

     accuracy                           0.98      1440
    macro avg       0.97      0.98      0.98      1440
 weighted avg       0.98      0.98      0.98      1440



9. Save the Model 

In [17]:
torch.save(model.state_dict(), 'car_damage_vit.pth')

metrics folder

In [20]:
import os
import pandas as pd
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure model is in eval mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(pixel_values=images).logits
        _, predicted = torch.max(outputs.data, 1)

        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Map back to filenames (make sure test_df is aligned with val_loader order)
all_filenames = test_df['filename'].values.tolist()

# Create directory to save metrics if it doesn't exist
output_dir = "metrics_output"
os.makedirs(output_dir, exist_ok=True)

# Save predictions to CSV
results_df = pd.DataFrame({
    'filename': all_filenames,
    'true_label': all_labels,
    'predicted_label': all_preds
})
results_csv_path = os.path.join(output_dir, "val_predictions.csv")
results_df.to_csv(results_csv_path, index=False)

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds) * 100

# Generate classification report dictionary and text
class_report = classification_report(all_labels, all_preds, target_names=class_names, output_dict=True)
class_report_text = classification_report(all_labels, all_preds, target_names=class_names)

# Save classification report text file
report_path = os.path.join(output_dir, "classification_report.txt")
with open(report_path, "w") as f:
    f.write(f"Validation Accuracy: {accuracy:.2f}%\n\n")
    f.write(class_report_text)

print(f"Validation Accuracy: {accuracy:.2f}%")
print(class_report_text)

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.tight_layout()

# Save confusion matrix plot
cm_path = os.path.join(output_dir, "confusion_matrix.png")
plt.savefig(cm_path)
plt.close()

print(f"Metrics saved in folder: {output_dir}")


Validation Accuracy: 97.99%
               precision    recall  f1-score   support

        crack       0.90      1.00      0.95        35
      scratch       0.98      0.97      0.98       474
    tire flat       1.00      0.97      0.99       104
         dent       0.97      0.99      0.98       413
glass shatter       1.00      0.99      0.99       233
  lamp broken       0.98      0.97      0.98       181

     accuracy                           0.98      1440
    macro avg       0.97      0.98      0.98      1440
 weighted avg       0.98      0.98      0.98      1440

Metrics saved in folder: metrics_output
